### code from https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/rag-intro.ipynb

In [1]:
# !wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-09-19 13:47:34--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py.2’

minsearch.py.2      100%[===================>]   3.74K  --.-KB/s    in 0.001s  

2024-09-19 13:47:34 (5.43 MB/s) - ‘minsearch.py.2’ saved [3832/3832]



In [1]:
import minsearch

In [2]:
import json

In [3]:
with open('documents.json','rt') as f_in:
    docs_raw = json.load(f_in)

In [4]:
documents =[]
for docs in docs_raw:
    for doc in  docs["documents"]: 
        doc["course"] =docs["course"]
        documents.append(doc)

In [5]:
documents[0].keys()

dict_keys(['text', 'section', 'question', 'course'])

In [6]:
from minsearch import Index

index = Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [7]:
index.fit(documents)

In [8]:
query = "The course has already started can i still enroll?"

filter_dict = {"course": "data-engineering-zoomcamp"}
boost_dict = {"question": 3, "section": 0.5}

results = index.search(
    query,
    filter_dict, 
    boost_dict
)

for result in results:
    print(result)

{'text': "Yes, even if you don't register, you're still eligible to submit.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.", 'section': 'General course-related questions', 'question': 'Course - Can I still join the course after the start date?', 'course': 'data-engineering-zoomcamp'}
{'text': 'Yes, we will keep all the materials after the course /finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.', 'section': 'General course-related questions', 'question': 'Course - Can I follow the course after it finishes?', 'course': 'data-engineering-zoomcamp'}
{'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nGit\nLook

In [9]:
prompt_template="""
You are a teaching assistant, Please answer the QUESTION based on facts from the CONTEXT, 
If CONTEXT doesnot have the facts, Please answer with NONE.

QUESTION: {question}

CONTEXT: {context}
""".strip()

In [10]:
from ai21 import AI21Client
from ai21.models.chat import UserMessage

# One way of passing your key to the client.
import os
AI21_API_KEY = os.environ["AI21_API_KEY"]
client = AI21Client(api_key=AI21_API_KEY)

def single_message_instruct(content):
    messages = [UserMessage(content=content)]
    response = client.chat.completions.create(
        model="jamba-1.5-large",
        messages=messages,
        top_p=1.0 # Setting to 1 encourages different responses each call.
    )
    return response.to_json()


In [11]:
response = single_message_instruct(query)

/tmp/ipykernel_18558/168767803.py:21: DeprecationWarning: The 'to_json' method is deprecated and will be removed in a future version. Please use Pydantic's built-in methods instead.
  return response.to_json()


In [12]:
import json
json_response = json.loads(response)
json_response["choices"][0]["message"]["content"]

"Whether you can still enroll in a course that has already started depends on the policies of the institution or organization offering the course. Here are some general steps you can take to find out:\n\n1. **Check the Course Website**: Look for information on enrollment deadlines and policies on the course's official website.\n\n\n2. **Contact the Registrar or Admissions Office**: If it's an academic institution, reach out to the registrar or admissions office. They can provide specific details about late enrollment.\n\n\n3. **Email the Course Instructor**: For more informal or online courses, you might be able to contact the instructor directly to ask if late enrollment is possible.\n\n\n4. **Visit the Course Office**: If the course is part of a local institution, visiting the office in person can sometimes yield quicker results.\n\n\n5. **Check Online Forums or FAQs**: Some courses have online forums or FAQ sections where such questions are addressed.\n\n\nWould you like help findin

In [13]:
def something_new(content):
    response = single_message_instruct(content)
    json_response = json.loads(response)
    content = json_response["choices"][0]["message"]["content"]
    return content

In [14]:
print(something_new(query))

Yes, you can still enroll in the course. Please follow these steps to complete your enrollment:

1. **Visit the Enrollment Page:** Go to the course enrollment page on the university's website.
2. **Fill Out the Application Form:** Complete the online application form with your personal and academic information.
3. **Submit Required Documents:** Upload necessary documents such as transcripts, identification, and any other required materials.
4. **Pay the Enrollment Fee:** Pay the enrollment fee through the university's payment portal.
5. **Confirmation:** Once you have completed these steps, you will receive a confirmation email with further instructions.

If you have any questions or need assistance, please contact the admissions office at [admissions office contact information].


/tmp/ipykernel_18558/168767803.py:21: DeprecationWarning: The 'to_json' method is deprecated and will be removed in a future version. Please use Pydantic's built-in methods instead.
  return response.to_json()


In [15]:
context =""
for doc in results:
    context =context+f"section: {doc['section']} \nquestion: {doc["question"]}\n answer: {doc["text"]} \n\n"
    

In [16]:
prompt = prompt_template.format(question=query , context=context).strip()

In [17]:
print(prompt)

You are a teaching assistant, Please answer the QUESTION based on facts from the CONTEXT, 
If CONTEXT doesnot have the facts, Please answer with NONE.

QUESTION: The course has already started can i still enroll?

CONTEXT: section: General course-related questions 
question: Course - Can I still join the course after the start date?
 answer: Yes, even if you don't register, you're still eligible to submit.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute. 

section: General course-related questions 
question: Course - Can I follow the course after it finishes?
 answer: Yes, we will keep all the materials after the course /finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project. 

section: General course-related questions 
question: Co

In [18]:
print(something_new(prompt))

Yes, even if you don't register, you're still eligible to submit. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.


/tmp/ipykernel_18558/168767803.py:21: DeprecationWarning: The 'to_json' method is deprecated and will be removed in a future version. Please use Pydantic's built-in methods instead.
  return response.to_json()


# modularization

In [24]:
# rag: retrieval 
def search(query):    
    filter_dict = {"course": "data-engineering-zoomcamp"}
    boost_dict = {"question": 3, "section": 0.5}
    
    results = index.search(
        query,
        filter_dict, 
        boost_dict
    )
    return results
search_results = search("The course has already started can i still enroll?")

In [26]:
# rag: augmentation 
def build_prompt(query,search_results):
    prompt_template="""
You are a teaching assistant, Please answer the QUESTION based on facts from the CONTEXT, 
If CONTEXT doesnot have the facts, Please answer with NONE.

QUESTION: {question}

CONTEXT: {context}
""".strip()
    context =""
    for doc in results:
        context =context+f"section: {doc['section']} \nquestion: {doc["question"]}\n answer: {doc["text"]} \n\n"
    prompt = prompt_template.format(question=query , context=context).strip()
    return prompt 
prompt = build_prompt(query,search_results)

In [29]:
# rag: generation 
from ai21 import AI21Client
from ai21.models.chat import UserMessage

# One way of passing your key to the client.
import os
AI21_API_KEY = os.environ["AI21_API_KEY"]
client = AI21Client(api_key=AI21_API_KEY)

def single_message_instruct(content):
    messages = [UserMessage(content=content)]
    response = client.chat.completions.create(
        model="jamba-1.5-large",
        messages=messages,
        top_p=1.0 # Setting to 1 encourages different responses each call.
    )
    return response.to_json()

def llm(prompt):
    response = single_message_instruct(prompt)
    json_response = json.loads(response)
    content = json_response["choices"][0]["message"]["content"]
    return content
llm(prompt)


/tmp/ipykernel_18558/1652257752.py:17: DeprecationWarning: The 'to_json' method is deprecated and will be removed in a future version. Please use Pydantic's built-in methods instead.
  return response.to_json()


"Yes, even if you don't register, you're still eligible to submit. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."

In [33]:
query = "How to run kafka?"
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query,search_results)
    answer = llm(prompt)
    return answer
rag(query)

/tmp/ipykernel_18558/1652257752.py:17: DeprecationWarning: The 'to_json' method is deprecated and will be removed in a future version. Please use Pydantic's built-in methods instead.
  return response.to_json()


'To run Kafka, you can follow these steps:\n\n1. **Install Kafka**: Download and extract Kafka from the official Apache Kafka website or use a package manager like Homebrew or apt-get.\n\n\n2. **Start Zookeeper**: Kafka uses Zookeeper for distributed coordination. Start Zookeeper by running the following command:\n\n\n```sh\nbin/zookeeper-server-start.sh config/zookeeper.properties\n```\n\n\n3. **Start Kafka Broker**: Start the Kafka broker by running the following command:\n\n\n```sh\nbin/kafka-server-start.sh config/server.properties\n```\n\n\n4. **Create a Topic**: Create a topic to store your messages. Run the following command:\n\n\n```sh\nbin/kafka-topics.sh --create --zookeeper localhost:2181 --replication-factor 1 --partitions 1 --topic test\n```\n\n\n5. **Produce Messages**: Start a producer to send messages to the topic. Run the following command:\n\n\n```sh\nbin/kafka-console-producer.sh --broker-list localhost:9092 --topic test\n```\n\n\n6. **Consume Messages**: Start a con